In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import atexit
from picamera import PiCamera
import RPi.GPIO as GPIO 
from time import sleep
from model.image_model import TrashModel
import os
import time

Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [3]:
class TrashApp:
    def __init__(self, dataset_path, model_weights_file, model=TrashModel(), pinout={
    'BTN_INPUT_PIN': 18,
    'LED_CTRL_OUTPUT_PIN': 25,
    'LED_RECYCLABLE_OUTPUT_PIN': 15,
    'LED_NON_RECYCLABLE_OUTPUT_PIN': 7
}):
        self.dataset_path = dataset_path
        self.model_weights_file = model_weights_file
        self.model = model
        self.model.load_weights(model_weights_file)
        self.pinout = pinout
        
        self._setup()
        
    def _cleanup_gpio(self):
        print('Cleaning up GPIO and exiting...')
        GPIO.cleanup()

    def _init_pins(self):
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.pinout['BTN_INPUT_PIN'], GPIO.IN, pull_up_down=GPIO.PUD_UP)
        GPIO.setup(self.pinout['LED_CTRL_OUTPUT_PIN'], GPIO.OUT)
        GPIO.setup(self.pinout['LED_RECYCLABLE_OUTPUT_PIN'], GPIO.OUT)
        GPIO.setup(self.pinout['LED_NON_RECYCLABLE_OUTPUT_PIN'], GPIO.OUT)
        
    def _setup(self):
        atexit.register(self._cleanup_gpio)
        self._init_pins()

        # Give feedback to user of camera initialization and warm up. 
        GPIO.output(self.pinout['LED_CTRL_OUTPUT_PIN'], GPIO.HIGH)

        self.camera = PiCamera()
        sleep(2)

        print('Camera ready!')
        GPIO.output(self.pinout['LED_CTRL_OUTPUT_PIN'], GPIO.LOW)
        GPIO.output(self.pinout['LED_RECYCLABLE_OUTPUT_PIN'], GPIO.LOW)
        GPIO.output(self.pinout['LED_NON_RECYCLABLE_OUTPUT_PIN'], GPIO.LOW)
        
    def run(self):

        if GPIO.input(self.pinout['BTN_INPUT_PIN']) == False:
            GPIO.output(self.pinout['LED_CTRL_OUTPUT_PIN'], GPIO.HIGH)
            filename = 'photo_' + str(int(time.time())) + '.jpg'
            self.camera.capture(filename)
            class_output = self.model.classifier(filename)
            print(class_output)
            GPIO.output(self.pinout['LED_CTRL_OUTPUT_PIN'], GPIO.LOW)
            if class_output == 'recyclable':
                GPIO.output(self.pinout['LED_RECYCLABLE_OUTPUT_PIN'], GPIO.HIGH)
                sleep(1)
                GPIO.output(self.pinout['LED_RECYCLABLE_OUTPUT_PIN'], GPIO.LOW)
                os.rename(filename, self.dataset_path + '/recyclable/' + filename)
            if class_output == 'nonrecyclable':
                GPIO.output(self.pinout['LED_NON_RECYCLABLE_OUTPUT_PIN'], GPIO.HIGH)
                sleep(1)
                GPIO.output(self.pinout['LED_NON_RECYCLABLE_OUTPUT_PIN'], GPIO.LOW)
                os.rename(filename, self.dataset_path + '/nonrecyclable/' + filename)

            sleep(.3)


In [4]:
trash_app = TrashApp('../data-collection', '../model/first_try.h5')

Camera ready!


In [5]:
while(True):
    trash_app.run()
    sleep(.1)

recyclable


KeyboardInterrupt: 